### Importação de configurações e funções

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import DeltaTable
import zipfile
import os
from pyspark.sql import DataFrame
from pyspark.sql.functions import current_timestamp
from pyspark.sql.functions import lit
 

In [0]:
import sys
sys.path.append("/Workspace/Users/kgenuins@emeal.nttdata.com/project-insight-lab-databricks")

from Config.spark_config import apply_storage_config
from Config.storage_config import *

apply_storage_config(spark)

In [0]:
input_path = f"{cnpj_path}"

# Aquisição de dados(Camada Bronze)

* Os dados originalmente são depositados no "blob" da Azure

* Então, os arquivos são lidos do "blob" e descompactados diretamente no cluster Databricks em um diretório abaixo do /tmp do cluster

* Os dados relacionados à CNPJ relevantes no momento são:
  * Empresas
  * Estabelecimentos
  * Simples
  * Sócios
  * Países
  * Municípios
  * Qualificações
  * Naturezas
  * CNAEs

Para a camada "Bronze", apenas os headers são adicionados além das colunas da data de ingestão e do path de origem dos dados






In [0]:
arquivos = dbutils.fs.ls(input_path)

In [0]:
# Descomprimindo os arquivos zip
input_path = f"{cnpj_path}"  
# Path temporário para os arquivos descompactados
output_path = "/dbfs/tmp/deszipados"  # pasta de destino (DBFS)

# Cria a pasta de destino se não existir
os.makedirs(output_path, exist_ok=True)

# Lista arquivos na pasta do DBFS
arquivos = dbutils.fs.ls(input_path)

def decompress_file(arquivo):
    """Descompassa um arquivo zip no DBFS.
    """
    if arquivo.name.endswith(".zip"):
        # Caminho do zip no DBFS
        caminho_zip_dbfs = arquivo.path

        # Caminho local temporário para descompactar
        caminho_zip_local = f"/tmp/{arquivo.name}"

        # Copia do DBFS para local
        dbutils.fs.cp(caminho_zip_dbfs, f"file:{caminho_zip_local}")

        # Cria uma subpasta para cada zip
        nome_subpasta = os.path.splitext(arquivo.name)[0]
        caminho_subpasta = os.path.join(output_path, nome_subpasta)
        os.makedirs(caminho_subpasta, exist_ok=True)

        # Descompacta
        with zipfile.ZipFile(caminho_zip_local, 'r') as zip_ref:
            zip_ref.extractall(caminho_subpasta)
        
        print(f"Descompactado {arquivo.name} em {caminho_subpasta}")

# Descomprime cada arquivo da lista 'arquivos'
[decompress_file(arquivo) for arquivo in arquivos]


# Listando os arquivos deszipados. Não relevante para o pipeline. Usar somente para depuração

In [0]:
display(dbutils.fs.ls(cnpj_path))

## Utilitários

In [0]:
def default_treatment_bronze_df(df: DataFrame) -> DataFrame:
    """ Tratamento padrão para DataFrames na camada Bronze. Cria uma coluna para gravar o caminho de origem dos dados e a data de ingestão.
    
    df: DataFrame a ser tratado.
    return: DataFrame tratado.
    
    """
    # Adicicionando a coluna ingestion_dt 
    df = df.withColumn("ingestion_dt", current_timestamp())
    # Criando uma coluna de particionamento chamada 'anomes', cujo conteúdo é extraido das informações 'ano' e 'mês' da coluna ingestion_dt
    df = df.withColumn("anomes", concat(year(df.ingestion_dt), month(df.ingestion_dt)))
    return df

In [0]:
# Arquivos de empresas
# Coluna dos arquivos de EMPRESAS
# TODO colocar a definição do esquema em um lugar mais apropriado a definir.
empresas_columns = [
    "CNPJ_BASICO", 
    "RAZAO_SOCIAL_NOME_EMPRESARIAL", 
    "NATUREZA_JURIDICA", 
    "QUALIFICACAO_DO_RESPONSAVEL",  
    "CAPITAL_SOCIAL_DA_EMPRESA",
    "PORTE_DA_EMPRESA",
    "ENTE_FEDERATIVO_RESPONSAVEL"
]
try:
    # Caminho de acesso para todos os arquivos descompactados de empresas
    companies_path = "dbfs:/tmp/deszipados/Empresas*/*.EMPRECSV"
    # Lendo e unificando os arquivos descompactados Empresas0,1,2,3...
    empresas_df = spark.read.csv(
        companies_path, 
        header=False, 
        encoding="iso-8859-1", sep=';'
    )
    
except Exception as e:
    print(f"Error: Não foi possível ler os dados de EMPRESAS no caminho '{companies_path}' - {e}")
    # Abortando a execução
    dbutils.notebook.exit()
    
# Iniciando a ingestão de dados de empresas
try:
    # Adicionando o 'header' com os nomes das colunas
    empresas_df = empresas_df.toDF(*empresas_columns)
    # Criando uma coluna para gravar o caminho de origem dos dados
    empresas_df = empresas_df.withColumn("origin_path_name", lit("dbfs:/tmp/deszipados/Empresas*/*.EMPRECSV"))
    # Tratamento padrão para as tabelas bronze
    empresas_df = default_treatment_bronze_df(empresas_df)
    # print(empresas_df.schema)
    display(empresas_df)

    # Gravando a Delta em modo 'overwrite' na camada "Bronze"
    (empresas_df
        .write
        .format("delta")
        .mode("overwrite")
        .partitionBy("anomes")
        .save(f"{bronze_path}/cnpj/Empresas")
    )
except Exception as e:
    print(f"Error: A ingestão dos dados de empresa em {bronze_path}/cnpj/Empresas. falhou! - {e}")
    dbutils.notebook.exit()


In [0]:
# Estabelecimentos
estabelecimentos_columns = [
    "CNPJ_BASICO",
    "CNPJ_ORDEM",
    "CNPJ_DV",
    "IDENTIFICADOR_MATRIZ_FILIAL",
    "NOME_FANTASIA",
    "SITUACAO_CADASTRAL",
    "DATA_SITUACAO_CADASTRAL",
    "MOTIVO_SITUACAO_CADASTRAL",
    "NOME_CIDADE_EXTERIOR",
    "PAIS",
    "DT_INICIO_ATIVIDADE",
    "CNAE_FISCAL_PRINCIPAL",
    "CNAE_FISCAL_SECUNDARIA",
    "TIPO_LOGRADOURO",
    "LOGRADOURO",
    "NUMERO",
    "COMPLEMENTO",
    "BAIRRO",
    "CEP",
    "UF",
    "MUNICIPIO",
    "DDD1",
    "TELEFONE1",
    "DDD2",
    "TELEFONE2",
    "DDD_FAX",
    "FAX",
    "CORREIO_ELETRONICO",
    "SITUACAO_ESPECIAL",
    "DATA_SITUACAO_ESPECIAL"
]

try:
    estabelecimentos_df = spark.read.csv("dbfs:/tmp/deszipados/Estabelecimentos*/*.ESTABELE", header=False, encoding="iso-8859-1", sep=';')
    # Adicionando o 'header' com os nomes das colunas
    estabelecimentos_df = estabelecimentos_df.toDF(*estabelecimentos_columns)
    # Criando uma coluna para gravar o caminho de origem dos dados
    estabelecimentos_df = estabelecimentos_df.withColumn("origin_path_name", lit("dbfs:/tmp/deszipados/Estabelecimentos*/*.ESTABELE"))
    # Tratamento padrão para as tabelas bronze
    estabelecimentos_df = default_treatment_bronze_df(estabelecimentos_df)
    display(estabelecimentos_df)
    # Gravando na camada bronze usando delta
    ( 
    estabelecimentos_df.write
        .format("delta")
        .mode("overwrite")
        .partitionBy("anomes")
        .save(f"{bronze_path}/cnpj/Estabelecimentos")
    )

except Exception as e:
    print(f"Error: A ingestão dos dados de estabelecimentos em {bronze_path}/cnpj/Estabelecimentos falhou! - {e}")
    dbutils.notebook.exit()


In [0]:
# Arquivos de simples
# Coluna dos arquivos de simples
simples_columns = [
    "CNPJ_BASICO", 
    "OPCAO_PELO_SIMPLES", 
    "DATA_DE_OPCAO_PELO_SIMPLES", 
    "DATA_DE_EXCLUSAO_DO_SIMPLES", 
    "OPCAO_PELO_MEI", 
    "DATA_DE_OPCAO_PELO_MEI", 
    "DATA_DE_EXCLUSAO_DO_MEI"
]
try:
    # Lendo e unificando os arquivos 
    simples_df = spark.read.csv("dbfs:/tmp/deszipados/Simples*", header=False, encoding="iso-8859-1", sep=';')
    simples_df = simples_df.toDF(*simples_columns)

    # Criando uma coluna para gravar o caminho de origem dos dados
    simples_df = simples_df.withColumn("origin_path_name", lit("dbfs:/tmp/deszipados/Simples*"))

    # Tratamento padrão para as tabelas bronze
    simples_df = default_treatment_bronze_df(simples_df)

    display(simples_df)
    # Gravando os dados na camada bronze
    (
        simples_df
        .write
        .format("delta")
        .mode("overwrite")
        .option("mergeSchema", "true")
        .save(f"{bronze_path}/cnpj/Simples")    
    )

except Exception as e:
    print(f"Error: A ingestão dos dados de simples em {bronze_path}/cnpj/Simples falhou! - {e}")
    dbutils.notebook.exit()


In [0]:
#Arquivo de sócios 

# Padronizando as colunas dos arquivos de sócios
socios_columns = [
    "CNPJ_BASICO", 
    "IDENTIFICADOR_DO_SOCIO", 
    "NOME_DO_SOCIO", 
    "CPF/CNPJ_DO_SOCIO", 
    "QUALIFICACAO_DO_SOCIO", 
    "DATA_DE_ENTRADA_NA_SOCIEDADE", 
    "PAIS", 
    "REPRESENTANTE_LEGAL", 
    "NOME_DO_REPRESENTANTE_LEGAL", 
    "QUALIFICACAO_DO_REPRESENTANTE_LEGAL",
    "FAIXA_ETARIA"
]

# Leitura dos arquivos CSV
socios_df = spark.read.csv(
    "dbfs:/tmp/deszipados/Socios*/*.SOCIOCSV",
    header=False,
    sep=";",
    encoding="iso-8859-1"
)

socios_df = socios_df.toDF(*socios_columns)

# Adicionando colunas de data de ingestão
socios_df = default_treatment_bronze_df(socios_df)

# Criando uma coluna para gravar o caminho de origem dos dados
socios_df = socios_df.withColumn("origin_path_name", lit("dbfs:/tmp/deszipados/Socios*/*.SOCIOCSV"))

#Gravando em delta.
try:
    (
        socios_df
            .write
            .format("delta")
            .mode("overwrite")
            .option("mergeSchema", "true")
            .save(f"{bronze_path}/cnpj/Socios")
    )
except Exception as e:
    print("Erro ao gravar Delta na camada Bronze")
    raise e

display(socios_df)

In [0]:
# Arquivos de Países

# Padronizando as colunas dos arquivos de paises
paises_columns = [
    "CODIGO",
    "DESCRICAO"
]

# Leitura dos arquivos CSV
paises_df = spark.read.csv("dbfs:/tmp/deszipados/Paises*", header=False, encoding="iso-8859-1", sep=';')

paises_df = paises_df.toDF(*paises_columns)

# Adicionando colunas de data de ingestão
paises_df = default_treatment_bronze_df(paises_df)

# Criando uma coluna para gravar o caminho de origem dos dados
paises_df = paises_df.withColumn("origin_path_name", lit("dbfs:/tmp/deszipados/Paises*"))

# Gravando em delta
try:
    (
        paises_df
            .write
            .format("delta")
            .mode("overwrite")
            .option("mergeSchema", "true")
            .save(f"{bronze_path}/cnpj/Paises")
    )
except Exception as e:
    print("Erro ao gravar Delta na camada Bronze")
    raise e
display(paises_df)


In [0]:
# Arquivos de Municipios

# Coluna dos arquivos de Municipios
municipios_columns = [
    "CODIGO",
    "DESCRICAO"
]

# Lendo e unificando os arquivos 
municipios_df = spark.read.csv("dbfs:/tmp/deszipados/Municipios*", header=False, encoding="iso-8859-1", sep=';')

municipios_df = municipios_df.toDF(*municipios_columns)

# Adicionando as colunas de data de ingestão
municipios_df = default_treatment_bronze_df(municipios_df)

# Criando uma coluna para gravar o caminho de origem dos dados
municipios_df = municipios_df.withColumn("origin_path_name", lit("dbfs:/tmp/deszipados/Municipios*"))

# Gravando em delta.
try:
    (
        municipios_df
            .write
            .format("delta")
            .mode("overwrite")
            .option("mergeSchema", "true")
            .save(f"{bronze_path}/cnpj/Municipios")
    )
except Exception as e:
    print("Erro ao gravar Delta na camada Bronze")
    raise e


display(municipios_df)


In [0]:
# Arquivos de Qualificações de empresas

# Coluna dos arquivos de Qualificações de empresas
qualificacoes_columns = [
    "CODIGO",
    "DESCRICAO"
]

# Lendo e unificando os arquivos 
qualificacoes_df = spark.read.csv("dbfs:/tmp/deszipados/Qualificacoes*", header=False, encoding="iso-8859-1", sep=';')

qualificacoes_df = qualificacoes_df.toDF(*qualificacoes_columns)

#Adicionando as colunas de data de ingestão
qualificacoes_df = default_treatment_bronze_df(qualificacoes_df)

# Criando uma coluna para gravar o caminho de origem dos dados
qualificacoes_df = qualificacoes_df.withColumn("origin_path_name", lit("dbfs:/tmp/deszipados/Qualificacoes*"))

#Gravando em delta.
try:
    (
        qualificacoes_df
            .write
            .format("delta")
            .mode("overwrite")
            .option("mergeSchema", "true")
            .save(f"{bronze_path}/cnpj/Qualificacoes")
    )
except Exception as e:
    print("Erro ao gravar Delta na camada Bronze")
    raise e


display(qualificacoes_df)


In [0]:
# Arquivos de Naturezas

# Coluna dos arquivos de Naturezas
naturezas_columns = [
    "CODIGO",
    "DESCRICAO"
]

# Lendo e unificando os arquivos 
naturezas_df = spark.read.csv("dbfs:/tmp/deszipados/Naturezas*", header=False, encoding="iso-8859-1", sep=';')

naturezas_df = naturezas_df.toDF(*naturezas_columns)

#Adicionando as colunas de data de ingestão
naturezas_df = default_treatment_bronze_df(naturezas_df)

# Criando uma coluna para gravar o caminho de origem dos dados
naturezas_df = naturezas_df.withColumn("origin_path_name", lit("dbfs:/tmp/deszipados/Naturezas*"))

#Gravando em delta.
try:
    (
        naturezas_df
            .write
            .format("delta")
            .mode("overwrite")
            .option("mergeSchema", "true")
            .save(f"{bronze_path}/cnpj/Naturezas")
    )
except Exception as e:
    print("Erro ao gravar Delta na camada Bronze")
    raise e


display(naturezas_df)


In [0]:
# Arquivos de Cnaes

# Coluna dos arquivos de Cnaes
cnaes_columns = [
    "CODIGO",
    "DESCRICAO"
]

# Lendo e unificando os arquivos 
cnaes_df = spark.read.csv("dbfs:/tmp/deszipados/Cnaes*", header=False, encoding="iso-8859-1", sep=';')
cnaes_df = cnaes_df.toDF(*cnaes_columns)

# Adicionando as colunas de data de ingestão
cnaes_df = default_treatment_bronze_df(cnaes_df)

# Criando uma coluna para gravar o caminho de origem dos dados
cnaes_df = cnaes_df.withColumn("origin_path_name", lit("dbfs:/tmp/deszipados/Cnaes*"))

#Gravando em delta.
try:
    (
        cnaes_df
            .write
            .format("delta")
            .mode("overwrite")
            .option("mergeSchema", "true")
            .save(f"{bronze_path}/cnpj/Cnaes")
    )
except Exception as e:
    print("Erro ao gravar Delta na camada Bronze")
    raise e


display(cnaes_df)


In [0]:
# Arquivos de Motivos

# Coluna dos arquivos de Motivos
motivos_columns = [
    "CODIGO",
    "DESCRICAO"
]

# Lendo e unificando os arquivos 
motivos_df = spark.read.csv("dbfs:/tmp/deszipados/Motivos*", header=False, encoding="iso-8859-1", sep=';')
motivos_df = motivos_df.toDF(*motivos_columns)

# Adicionando as colunas de data de ingestão
motivos_df = default_treatment_bronze_df(motivos_df)

# Criando uma coluna para gravar o caminho de origem dos dados
motivos_df = motivos_df.withColumn("origin_path_name", lit("dbfs:/tmp/deszipados/Motivos*"))

#Gravando em delta.
try:
    (
        motivos_df
            .write
            .format("delta")
            .mode("overwrite")
            .option("mergeSchema", "true")
            .save(f"{bronze_path}/cnpj/Motivos")
    )
except Exception as e:
    print("Erro ao gravar Delta na camada Bronze")
    raise e


display(motivos_df)


In [0]:
#%run /Workspace/Users/kgenuins@emeal.nttdata.com/project-insight-lab-databricks/Bronze/ingestao_incremental_cnpj

In [0]:
#processar_csv_bronze_incremental_naturezas

In [0]:


#naturezas_df, qtd = processar_csv_bronze_incremental_naturezas(
#    input_path="dbfs:/tmp/deszipados/Naturezas*",
#    output_path=f"{bronze_path}/cnpj/Naturezas",
#    colunas=["CODIGO", "DESCRICAO"]

print(f"Foram inseridos {qtd} registros novos")
